In [42]:
!pip list --format=freeze > /content/drive/MyDrive/Colab Notebooks/EE-451/requirements.txt

In [43]:
# Install
!pip install colorthief

# Import
import numpy as np
from typing import Union
from glob import glob
import os
from PIL import Image
import matplotlib.pyplot as plt
from colorthief import ColorThief
from skimage import io
import cv2
from skimage.filters import gabor_kernel, gabor
from scipy import ndimage as ndi
from skimage.util import img_as_float
from sklearn.decomposition import PCA
import re
from sklearn.preprocessing import StandardScaler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# [IAPR][iapr]: Project


**Group ID:** 37

**Author 1 (sciper):** Elias De Smijter (366670)  
**Author 2 (sciper):** Félicie Alice Agnès Marie Giraud-Sauveur (284220)   
**Author 3 (sciper):** Cyril Felix Monette (299554)   

**Release date:** 27.04.2023


## Important notes

The assignments are designed to teach practical implementation of the topics presented during class as well as preparation for the final project, which is a practical project which ties together the topics of the course. 

As such, in the lab assignments/final project, unless otherwise specified, you may, if you choose, use external functions from image processing/ML libraries like opencv and sklearn as long as there is sufficient explanation in the lab report. For example, you do not need to implement your own edge detector, etc.

**! Before handling back the notebook !** rerun the notebook from scratch `Kernel` > `Restart & Run All`


[iapr]: https://github.com/LTS5/iapr

---
## 0. Introduction

In this project, you will be working on solving tiling puzzles using image analysis and pattern recognition techniques. Tiling puzzles are a classic type of puzzle game that consists of fitting together pieces of a given shape (in this case squared to form a complete image. The goal of this project is to develop an algorithm that can automatically reconstruct tiling puzzles from a single input image. 

---

## 1. Data

### Input data
To achieve your task, you will be given images that look like this:


![train_00.png](data_project/project_description/train_00.png)

### Example puzzle content
Example of input of solved puzzles. 
Solution 1
<img src="data_project/project_description/solution_example.png" width="512"/>
Solution 2
<img src="data_project/project_description/solution_example2.jpg" width="512"/>


### 1.1. Image layout

- The input for the program will be a single image with a size of __2000x2000 pixels__, containing the pieces of the tiling puzzles randomly placed in it. The puzzles sizes vary from __3x3, 3x4, or 4x4__ size. 
    -__You are guaranteed to always have the exact number of pieces for each puzzle__ 
        -For each puzzle you always are expected to find exaclty 9,12,16 pieces
        -If you find something else, either you are missing pieces, or added incorrect pieces for the puzzle

- The puzzle pieces are square-shaped with dimensions of 128x128 pixels (before rotation). 

- The input image will contain pieces from __two or three (but never four)__ different tiling puzzles, as well as some __extra pieces (outliers)__ that do not belong to either puzzle.


## 2. Tasks (Total 20 points) 


The project aims to:
1) Segment the puzzle pieces from the background (recover the pieces of 128x128 pixels)   \[ __5 points__ \] 

2) Extract features of interest from puzzle pieces images \[ __5 points__ \]   

3) Cluster puzzle pieces to identify which puzzle they belong, and identify outliers.  \[ __5 points__ \]   

4) Solve tiling puzzle (find the rotations and translations to correctly allocate the puzzle pieces in a 3x3, 3x4 or 4x4 array.) \[ __5 points__ \]   

##### The images used for the puzzles have self-repeating patterns or textures, which ensures that all puzzle pieces contain more or less the same features regardless of how they were cut. 




### 1.2. Output solution pieces.

For each inpute image, the output solution will include N images with solved puzzles, where N is the number of puzzles in the input image. and M images, that are Each of these images will contain the solved solution to one of the N puzzles in the input. 


-  Example input:  train_05.png

- Example solution:
        -solution_05_00.png solution_05_01.png solution_05_02.png 
        -outlier_05_00.png outlier_05_01.png outlier_05_02.png ...

- Example input:  train_07.png
- Example solution:
        -solution_07_00.png solution_07_01.png 
        -outlier_07_00.png outlier_07_01.png outlier_07_02.png ...


__Watch out!__ output resolution should always be like this:  
<table ><tr><th >Puzzle pieces <th><th> pixel dimentions <th> <th> pixel dimentions <th> <tr>
<tr><td> 3x3 <td><td> 384x384 <td><td> 3(128)x3(128) <td> <tr>
<tr><td> 3x4 <td><td> 384x512 <td><td> 3(128)x4(128)<tr>
<tr><td> 4x4 <td><td> 512x512 <td><td> 4(128)x4(128)<tr>
<tr><td> 1x1 (outlier)<td><td> 128x128 <td><td> (1)128x(1)128 <td><tr><table>





__Order of the solutions (and rotations) it's not a problem for the grading__




the output solution will be a final image of resolution (1283)x(1283), with each piece correctly placed in its corresponding location in the 3x3 array. Similarly, if the puzzle consists of 3x4 or 4x4 pieces, the output solution will be an image of resolution (1283)x(1284) or (1284)x(1284)



### 1.3 Data folder Structure

You can download the data for the project here: [download data](https://drive.google.com/drive/folders/1k3xTH0ZhpqZb3xcZ6wsOSjLzxBNYabg3?usp=share_link)

```
data_project
│
└─── project_description
│    │    example_input.png      # example input images
│    │    example_textures1.png      # example input images
│    │    example_textures2.png      # example input images
│    └─── ultimate_test.jpg   # If it works on that image, you would probably end up with a good score
│
└─── train
│    │    train_00.png        # Train image 00
│    │    ...
│    │    train_16.png        # Train image 16
│    └─── train_labels.csv    # Ground truth of the train set
|    
└────train_solution
│    │    solution_00_00.png        # Solution puzzle 1 from Train image 00
│    │    solution_00_01.png        # Solution puzzle 2 from Train image 00
│    │    solution_00_02.png        # Solution Puzzle 3 from Train image 00
│    │    outlier_00_00.png         # outlier     from Train image 00
│    │    outlier_00_01.png         # outlier     from Train image 00
│    │    outlier_00_03.png         # outlier     from Train image 00
│    │    ...
│    │    solution_15_00.png        # Solution puzzle 1 from Train image 15
│    │    solution_15_01.png        # Solution puzzle 2 from Train image 15
│    │    outlier_15_00.png         # outlier     from Train image 15
│    └─── outlier_15_01.png         # outlier     from Train image 15
│
└─── test
     │    test_00.png         # Test image 00 (day of the exam only)
     │    ...
     └─── test_xx.png             # Test image xx (day of the exam only)
```



## 3. Evaluation

**Before the exam**
   - Create a zipped folder named **groupid_xx.zip** that you upload on moodle (xx being your group number).
   - Include a **runnable** code (Jupyter Notebook and external files) and your presentation in the zip folder.
   
**The day of the exam**
   - You will be given a **new folder** (test folder) with few images, but **no ground truth** (no solutions).
   - We will ask you to run your pipeline in **real time** and to send us your prediction of the task you obtain with the provided function **save_results**. 
   - On our side, we will compute the performance of your classification algorithm. 
   - To evaluate your method, we will use the **evaluate_solution** function presented below. To understand how the provided functions work, please read the documentation of the functions in **utils.py**.
   - **Please make sure your function returns the proper data format to avoid points penalty on the day of the exam**. 
---


## 4. Your code

### 4.1. Setup

In [44]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [45]:
# Import
import os 
import sys
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch

In [46]:
# Path folder
ROOT_PATH = "/content/drive/MyDrive/Colab Notebooks/EE-451"
print(os.listdir(ROOT_PATH))

['data_project', '__pycache__', 'project_colab.ipynb', 'customColorthief.py', 'rotated_rect_crop.py']

In [47]:
# Load images

def load_input_images(folder ="test"):
    imgnames = os.listdir(ROOT_PATH+"/data_project/"+folder)    
    images= [np.array(Image.open(os.path.join(ROOT_PATH+"/data_project/", folder, imgname)).convert('RGB')) for imgname in imgnames ]
    
    return images

### 4.2. Segmentation

In [48]:
# Check GPU
!nvidia-smi

Thu Jun  1 20:11:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    40W / 300W |  14938MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [49]:
# Install Segment Anything Model (SAM) and other dependencies

# The following software (Segment Anything) is taken from the Facebook Research team, i.e Kirillov, Alexander and Mintun, Eric and Ravi,
# Nikhila and Mao, Hanzi and Rolland, Chloe and Gustafson, Laura and Xiao, Tete and Whitehead, Spencer and Berg, Alexander C. and Lo, Wan-Yen
# and Doll'r, Piotr and Girshick, Ross. Available at https://github.com/facebookresearch/segment-anything

HOME = os.getcwd()
%cd {HOME}
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

# Download SAM weights

%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/content/weights
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-z_ozgi1j
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-z_ozgi1j
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
/content/weights
/content/weights/weights


/content/weights/weights/sam_vit_h_4b8939.pth ; exist: True

In [50]:
# Load SAM Model

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [51]:
# Generate masks generator with SAM

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.95,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=15000,  # Requires open-cv to run post-processing
)

In [52]:
# Import function to rotate rectangle and crop into a new image

%load_ext autoreload
%autoreload 2

sys.path.append(ROOT_PATH)
# The following is imported from an ad-hoc library made by Lei Mao, University of Chicago (3/1/2018) and is 
# available at https://github.com/leimao/Rotated-Rectangle-Crop-OpenCV/blob/master/rotated_rect_crop.py
from rotated_rect_crop import crop_rotated_rectangle

# image is the image matrix
# rect is the rotated rect object in OpenCV, i.e. (center (x,y), (width, height), angle of clock-wise rotation)
# center x is regarding to the width
# center y is regarding to the height
# width is the number of the columns of the rectangle.
# height is the number of the rows of the rectangle.
# The angle vector pointing to the "north" is exactly 0 degre

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
# Do the segmentation and get the mask for each train image 

best_masks = []

images = load_input_images(folder="test")

for i, image_bgr in enumerate(images):

  print('###########################################################################')
  print('############################# Image {} ####################################'.format(str(i).zfill(2)))
  print('###########################################################################')

  print('\n------------ 1. Sam_result ------------\n')

  # Generate masks generator with SAM
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  sam_result = mask_generator.generate(image_rgb)

  print('\n--------------- 2. Masks ---------------\n')

  # Get masks
  masks = [mask['segmentation'] for mask in sorted(sam_result, key=lambda x: x['area'], reverse=True)]
  

  print('\n------------- 3. Best mask -------------\n')

  # Keep mask with all pieces of the puzzle and visualize

  best_mask = masks[0]
  next_add=1
  while np.sum(best_mask)<3e6:
    best_mask = best_mask + masks[next_add]
    next_add = next_add +1

  best_masks.append(best_mask)
  print(f"best_mask saved as a composition of {next_add} mask(s)! \n \n")

###########################################################################

############################# Image 00 ####################################

###########################################################################

------------ 1. Sam_result ------------

--------------- 2. Masks ---------------

...

###########################################################################

------------ 1. Sam_result ------------

--------------- 2. Masks ---------------

------------- 3. Best mask -------------

best_mask saved as a composition of 1 mask(s)!

In [54]:
# Check the best_mask for each image, post-process and save them
from skimage.morphology import disk, binary_erosion

best_masks_save=np.copy(best_masks)
eroded_masks=np.zeros_like(best_masks_save)

for i, mask in enumerate(best_masks_save):
  mask_pp = cv2.convertScaleAbs(np.where(mask==True, 0, 1))
  mask_pp = binary_erosion(mask_pp, footprint=disk(20))
  eroded_masks[i]=mask_pp
  name = f"/data_project/masks/mask_{str(i).zfill(2)}.png"
  Image.fromarray(mask_pp).save(ROOT_PATH+name)

In [55]:
# Extract contours for each train image

all_contours = []

for i,mask in enumerate(eroded_masks):

  print('###########################################################################')
  print('############################# Image {} ####################################'.format(str(i).zfill(2)))
  print('###########################################################################')

  print('\n------------- 4. Contours -------------\n')

  # Use the best_mask to threshold the image and find the contours
  thresholded_image = cv2.convertScaleAbs(np.where(mask==True, 255, 0))
  contours, _ = cv2.findContours(thresholded_image, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
  all_contours.append([contour  for contour in contours if (cv2.contourArea(contour) > 8000) and (cv2.contourArea(contour) < 18000)])
  print('contours saved! \n \n')


###########################################################################

############################# Image 00 ####################################

###########################################################################

------------- 4. Contours -------------

contours saved!

...

###########################################################################

############################# Image 09 ####################################

###########################################################################

------------- 4. Contours -------------

contours saved!

In [56]:
# Transform contours to rectangle for each piece of puzzle for each train image

def rectangle(contours):

  rectangles = []

  for c in range(len(contours)):
    
    try:
      # Get the minimum area rectangle
      rect = cv2.minAreaRect(contours[c])

      # Adjust the size of the rectangle to match the target size (128x128 pixels)
      desired_size = (128, 128)
      adjusted_rect = (rect[0], desired_size, rect[2])
      rectangles.append(adjusted_rect)

      # Get the rectangle points
      #box = cv2.boxPoints(rect)
      #box = np.int0(box)

      # Prepare the source and destination points for the affine transformation
      #src_pts = box.astype(np.float32)
      #dst_pts = cv2.boxPoints(adjusted_rect).astype(np.float32)

      # Find the affine transformation matrix
      #M = cv2.getAffineTransform(src_pts[:3], dst_pts[:3])

      # Apply the transformation matrix to the rectangle points
      #adjusted_box = cv2.transform(np.array([box]), M)[0]

      # Round the coordinates to integers
      #adjusted_box = np.int0(adjusted_box)

    # Draw the adjusted rectangles on the image
    #plt.figure(figsize = (50,50))
    #plt.imshow(cv2.drawContours(image_bgr.copy(), [adjusted_box], 0, (0, 255, 0), 2))

    except Exception as error:
      print('issue with contour {}: '.format(c))
      print(error)
      continue

  return rectangles



all_rectangles = []
for i, contour in enumerate(all_contours):

  print('###########################################################################')
  print('############################# Image {} ####################################'.format(str(i).zfill(2)))
  print('###########################################################################')

  print('\n------------- 5. Rectangles -------------\n')
  
  # Get the list of rectangles from contours and visualize results
  rectangles = rectangle(contour)
  all_rectangles.append(rectangles)
  print("rectangles saved!")

###########################################################################

############################# Image 00 ####################################

###########################################################################

------------- 5. Rectangles -------------

rectangles saved!

...

###########################################################################

############################# Image 09 ####################################

###########################################################################

------------- 5. Rectangles -------------

rectangles saved!

In [57]:
# Extract+save each piece of puzzle for each train image
from rotated_rect_crop import crop_rotated_rectangle

for i,imagebgr in enumerate(images):

  print('###########################################################################')
  print('############################# Image {} ####################################'.format(str(i).zfill(2)))
  print('###########################################################################')

  print('\n-------------- 6. Cropping  -------------\n')
  
  # Get each pieces from the original image
  all_pieces = []
  for r in range(len(all_rectangles[i])):
    try:
      piece = crop_rotated_rectangle(imagebgr.copy(), all_rectangles[i][r], all_contours[i][r])
    except Exception as error:
      pass
    all_pieces.append(piece)
  print("all pieces added!")

  print('\n--------------- 7. Saving  -------------\n')
  
  # Save results
  for num_p, p in enumerate(all_pieces):
    try:
      name = '/data_project/pieces/train{}_piece{}.png'.format(str(i).zfill(2), num_p)
      Image.fromarray(p).save(ROOT_PATH+name)
    except Exception as error:
      print('Saving issue with train image {} piece {}: '.format(str(i).zfill(2), num_p))
      print(error)
      continue
  print("all pieces saved!")

###########################################################################

############################# Image 00 ####################################

###########################################################################

-------------- 6. Cropping  -------------

44
13
53
53
57
57
32
46
62
34
40
36
59
51
30
47
32
21
52
63
45
10
47
4
62
61
25
37


...

-------------- 6. Cropping  -------------

15
35
37
11
59
62
6
6
50
14
33
47
58
46
45
52
Proposed rectangle is not fully in the image.
Problem fixed
57
24
62
39
12
62
26
63
22
16
55
Proposed rectangle is not fully in the image.
Problem fixed
61


all pieces added!

--------------- 7. Saving  -------------

all pieces saved!

### 4.3. Features extraction

In [58]:
from scipy.fft import fft

def fourier_descr(im, descr_to_keep):
    im_ft_v = fft(im, axis=0)
    m_v = np.mean(im_ft_v, axis=0)

    im_ft_h = fft(im, axis=1)
    m_h = np.mean(im_ft_h, axis=1)

    return [int(i) for i in list(abs(m_h))[:descr_to_keep]+list(abs(m_v))[:descr_to_keep]]

In [59]:
# Function to load pieces

def load_piece(image_index, piece_index, folder="pieces", path=ROOT_PATH+"/data_project"):
    
    filename = "train{}_piece{}.png".format(image_index.zfill(2), piece_index)
    path_piece = os.path.join(path, folder, filename)
    
    piece = Image.open(path_piece).convert('RGB')
    piece = np.array(piece)
    return piece, path_piece

#### Function to get feature vector and apply it to each piece

In [60]:
from colorsys import rgb_to_hsv
from sklearn.decomposition import PCA
'''
Function that builds a list of features for the input image.
Image should be rgb for feature extraction to work!
'''
def get_features(image_index, piece_index):
    feature_vector = []

    color_thief = ColorThief(load_piece(image_index,piece_index)[1])
    dominant_colors = np.array(color_thief.get_palette(color_count = 2)[0:2]).reshape(2,3)
    for dominant_color in np.array(dominant_colors)/255:
        dominantHSV = 359*np.array(rgb_to_hsv(dominant_color[0],dominant_color[1], dominant_color[2]))
        feature_vector.extend(dominantHSV)

    im = np.array(load_piece(image_index,piece_index)[0])
    im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
     
    for theta in range(4):
        theta = theta / 4. * np.pi
        for sigma in (1,3):
            for frequency in (0.05, 0.25):
                filt_real, filt_im = gabor(image=im, frequency=frequency, theta=theta, sigma_x=sigma, sigma_y=sigma)
                amplitude = abs(filt_real+i*filt_im)
                feature_vector.append(amplitude.mean())
                feature_vector.append(amplitude.var())

    f_descr = fourier_descr(im, 3) # Keep the 3 first fourrier descriptors (= 6 descriptors because horizontal and vertical fft)
    feature_vector.extend(f_descr)

    return feature_vector

In [61]:
# Get a list of list of tuples to get the ref of each piece
# Ex: [[('00', '1'), ('00', '2'), ('00', '3')], [('01', '1'), ('01', '2'), ('01', '3')], etc.]

files_pieces = os.listdir(ROOT_PATH+"/data_project/pieces")
list_pieces = sorted([re.findall(r'train(\d+)_piece(\d+)\.png', s)[0] for s in files_pieces])

sublists = {}
for p in list_pieces:
    key = p[0]
    if key in sublists:
        sublists[key].append(p)
    else:
        sublists[key] = [p]

list_im_pieces = list(sublists.values())

In [62]:
# Apply get_features to each piece of each image

features_pieces = []
for puzzle in list_im_pieces:
  puzzle_features = []
  for piece in puzzle:
      features_piece=get_features(piece[0],piece[1])
      puzzle_features.append(features_piece)
  features_pieces.append(puzzle_features)
  print("Puzzle finished")

Puzzle finished

Puzzle finished

Puzzle finished

Puzzle finished

Puzzle finished

Puzzle finished

Puzzle finished

Puzzle finished

Puzzle finished

Puzzle finished

### 4.4. Clustering with KMeans

In [63]:
# Import
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [64]:
list_im_clusters = []

for idx_im in [idx_im for idx_im in range(len(features_pieces))]:

  scaler = StandardScaler()
  normalized_data = scaler.fit_transform(features_pieces[idx_im])

  # 1. Find the best k

  silhouette_scores = []
  k_range = range(2, 7)  # Range of k values to try

  for k in k_range:
      kmeans = KMeans(n_clusters=k, n_init='auto')
      labels = kmeans.fit(normalized_data).labels_
      silhouette_avg = silhouette_score(normalized_data, labels)
      silhouette_scores.append(silhouette_avg)

  best_k = k_range[silhouette_scores.index(max(silhouette_scores))] # Find the k value with the highest silhouette score

  # 2. Cluster with KMeans
  
  kmeans = KMeans(n_clusters=best_k, n_init='auto')
  kmeans.fit(normalized_data)
  
  clusters = kmeans.labels_
  list_im_clusters.append(list(clusters))

  clusters_centers = kmeans.cluster_centers_

In [65]:
# Group filenames by cluster

def group_clusters(tup_names, clusters):
  sublists = {}
  for i, tup_name in enumerate(tup_names):
      cluster = clusters[i]
      if cluster in sublists:
          sublists[cluster].append("train{}_piece{}.png".format(tup_name[0], tup_name[1]))
      else:
          sublists[cluster] = ["train{}_piece{}.png".format(tup_name[0], tup_name[1])]
  result = list(sublists.values())
  return result

filename_and_clusters = []

for i, l in enumerate(list_im_pieces):
  filename_and_clusters.append(group_clusters(list_im_pieces[i], list_im_clusters[i]))

In [66]:
# Function to see cluster per image

def see_clusters_per_image(im_idx):
  
  for cluster, l in enumerate(filename_and_clusters[im_idx]):

    fig, axs = plt.subplots(int(len(l)/5)+1, 5, sharey=True, figsize=(14,8))
    axs = axs.flatten()
    
    for ax, f_name in zip(axs, l):
      
      piece = Image.open(os.path.join(ROOT_PATH+"/data_project", "pieces", f_name)).convert('RGB')
      ax.imshow(piece)
      ax.set_title(f'cluster{cluster}')

  plt.show()

In [67]:
def get_clusters_per_image(im_idx):
  cluster_list = []
  for cluster, l in enumerate(filename_and_clusters[im_idx]):

    pieces_in_clusted = []
    
    for ax, f_name in zip(axs, l):
      piece = Image.open(os.path.join(ROOT_PATH+"/data_project", "pieces", f_name)).convert('RGB')
      pieces_in_clusted.append(np.uint8(piece))
    
    cluster_list.append(pieces_in_clusted)

  return cluster_list

In [68]:
###
from rich import print
from rich.progress import track
from rich.console import Console
###

def export_solutions(image_index , solutions , path = "data_project"  , group_id = "00"):
    """
    Wrapper funciton to load image and save solution

    solutions :
        image_index : index of the image to solve

        list with the following items
        solutions [0] = segmented mask of the puzzle (matrix of 2000x2000 dimentions) , 0 for background, 1 for puzzle piece 
        
        solutions [1] = matrix containing the features of the puzzles.  if there were  N pieces in the puzzle, and you extracted M Features per puzzle piece, then the feature map should be of size N x M

        solutions [2] =  list of lists of images, each list of images is a cluster of puzzle pieces. (it includes outliers as the last elementof the list)
                        If there are k clusters, then the list should have k elements, each element is a list
                        e.g. 

                    solution [2] [0] =  [cluster0_piece0 , cluster0_piece1 ...]
                    solution [2] [1] =  [cluster1_piece0 , cluster1_piece1 ...]
                    solution [2] [2] =  [cluster2_piece0 , cluster2_piece1 ...]
                    ....
                    solution [2] [k]   =  [clusterk_piece0 , clusterk_piece1 ... ]
                    solution [2] [k+1] = [ outlier_piece0, outlier_piece1 ...]
                        
    solutions [3] = list of images containing the puzzles
                    e.g.
                    solution [3] [0] =  solved_puzzle0 (image of 128*3 x 128*4)
                    solution [3] [1] =  solved_puzzle1 (image of 128*4 x 128*4)
                    solution [3] [2] =  solved_puzzle2 (image of 128*3 x 128*3)
                    ....



        folder : folder where the image is located, the day of the exam it will be "test"
        path : path to the folder where the image is located

        group_id : group id of the team
            
    ----------
    image:
        index number of the dataset

    Returns
    """
    
    saving_path = os.path.join(path , "solutions_group_" + str(group_id) )
    if not os.path.isdir(saving_path):
        os.mkdir(saving_path)

    print("saving solutions in folder: " , saving_path)

   
    ## call functions to solve image_loaded
    save_mask           (image_index , solutions[0] , saving_path)
    save_feature_map    (image_index , solutions[1] , saving_path)
    save_cluster        (image_index , solutions[2] , saving_path)
    save_solved_puzzles (image_index , solutions[3] , saving_path)

    
    return None

def save_mask(image_index , solution, saving_path):
    
    filename = os.path.join(saving_path, f"mask_{str(image_index).zfill(2)}.png")
    if solution.shape[0] != 2000 or solution.shape[1] != 2000:
        print("error in mask:  shape of image" , solution.shape)
        return
    if np.max(solution) ==1:
        solution = solution*255
    solution = np.array(solution , dtype = np.uint8)
    Image.fromarray(solution).save(filename)
1
def save_feature_map(image_index , solution, saving_path):
    filename = os.path.join(saving_path, f"feature_map_{str(image_index).zfill(2)}.txt")
    np.savetxt(filename , solution)

    #min max into 0 ,255 interval
    solution = (solution - np.min(solution)) / (np.max(solution) - np.min(solution))
    solution = np.array(solution*255 , dtype = np.uint8)
    filename = filename.replace(".txt" , ".png")
    Image.fromarray(solution).save(filename)
    
def save_cluster(image_index , solution, saving_path):

    
    filename = os.path.join(saving_path, f"cluster_images_{str(image_index).zfill(2)}.png")

    n_clusters = len(solution)
    len_clusters = [len(cluster) for cluster in solution]


    xlen = n_clusters*128
    ylen = np.max(len_clusters)*128
    
    
    whole_image = np.zeros((xlen ,ylen , 3) , dtype = np.uint8)

    for i in range(n_clusters):
        for j in range(len_clusters[i]):
            if solution[i][j].shape[0] != 128 or solution[i][j].shape[1] != 128:
                print("error in shape of image" , solution[i][j].shape)
                return
            whole_image[i*128:(i+1)*128 , j*128:(j+1)*128 , :] = solution[i][j]
    
    Image.fromarray(whole_image).save(filename)

def save_solved_puzzles(image_index , solution, saving_path):

    n_solutions = len(solution)

    for i , sol in enumerate(solution):
        print(sol.shape)
        sol = np.array(sol , dtype = np.uint8)
        filename = os.path.join(saving_path, f"solved_puzzle_{str(image_index).zfill(2)}_{str(i).zfill(2)}.png")
        Image.fromarray(sol).save(filename)


In [69]:
for i,features in enumerate(features_pieces):
  save_feature_map(i, features, ROOT_PATH+"/data_project/features/")


In [70]:
for i,image in enumerate(list_im_clusters):
  save_cluster(i,get_clusters_per_image(i),ROOT_PATH+"/data_project/clusters/")